# Rank repartition

This notebook is intended for exploring the data fetched from the riot games api in the [Data collection Notebook](data_collection.ipynb).

This will be my first time using intensively a vizualisation library in python, I will be using [Vega Altair](https://altair-viz.github.io/) since it integrates really well with notebooks from what I heard. Also it is build around pandas Dataframe so I hope to get more knowledge of pandas by working with vega altair.

In [64]:
import sqlite3
import os
import pandas as pd
import altair as alt


def query_player_count_by_rank(db_path, min_no_of_games=0):
    """
    Queries the number of players grouped by rank from the database.
    """

    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found: {db_path}")

    conn = sqlite3.connect(db_path)
    
    try:
        cursor = conn.cursor()
        cursor.execute(
            f"""
            SELECT tier, rank, COUNT(*), wins, losses
            FROM ranked_players
            WHERE wins + losses >= {min_no_of_games}
            GROUP BY tier, rank
            ORDER BY tier, rank 
            """
        )
        count = cursor.fetchall()
    except sqlite3.Error as e:
        print(f"An error occurred while querying the database: {e}")
        count = []
        
    finally:
        # Close the database connection
        conn.close()
        return count
    
def graph_player_count_by_rank(db_path, min_no_of_games=0):
    """
    Graphs the number of players grouped by rank from the database.
    """
    players_by_rank = query_player_count_by_rank(db_path, min_no_of_games)

    players_by_rank_dict = { f"{rank[0]} {rank[1]}": rank[2] for rank in players_by_rank }
    rank_ordering = [
        "BRONZE IV", "BRONZE III", "BRONZE II", "BRONZE I",
        "SILVER IV", "SILVER III", "SILVER II", "SILVER I",
        "GOLD IV", "GOLD III", "GOLD II", "GOLD I",
        "PLATINUM IV", "PLATINUM III", "PLATINUM II", "PLATINUM I",
        "EMERALD IV", "EMERALD III", "EMERALD II", "EMERALD I",
        "DIAMOND IV", "DIAMOND III", "DIAMOND II", "DIAMOND I",
    ]
    
    ordered_counts = [
        { 
            "tier_rank": key, 
            "count": players_by_rank_dict.get(key, 0),
        } for key in rank_ordering
    ]

    data = pd.DataFrame(ordered_counts)
    description = data.describe()
    
    print("Data Description:")
    print(description)

    print("Total Players:", data['count'].sum())

    bars_chart = (
        alt.Chart(data)
        .mark_bar(
            cornerRadiusTopLeft=10,
            cornerRadiusTopRight=10,
        )
        .encode(
            x=alt.X('tier_rank', sort=rank_ordering),  # Explicitly set the sort order
            y='count',
            color=alt.Color('tier_rank', 
                scale=alt.Scale(domain=rank_ordering, range=[
                    "#cd7f32", "#cd7f32", "#cd7f32", "#cd7f32",
                    "#c0c0c0", "#c0c0c0", "#c0c0c0", "#c0c0c0",
                    "#ffd700", "#ffd700", "#ffd700", "#ffd700",
                    "#e5e4e2", "#e5e4e2", "#e5e4e2", "#e5e4e2",
                    "#50C878", "#50C878", "#50C878", "#50C878",
                    "#b9fbc0", "#b9fbc0", "#b9fbc0", "#b9fbc0",
                ]),
                legend=None,
            ),
            tooltip=['tier_rank', 'count'],
        )
        .properties(
            width=1000,
            height=400,
        )
    )

    player_percent_chart = (
        alt.Chart(data)
        .mark_line(
            color="red",
            strokeWidth=2,
            interpolate='basis',
            clip=True,
        )
        .transform_window(
            # The field to average
            rolling_mean='mean(count)',
            # The number of values before and after the current value to include.
            frame=[-1, 1]
        ).encode(
            x=alt.X('tier_rank', sort=rank_ordering),  # Explicitly set the sort order
            y='rolling_mean:Q',
            tooltip=[
                alt.Tooltip('tier_rank', title='Rank'),
                alt.Tooltip('rolling_mean:Q', title='Average Players'),
            ],
        ).properties(
            width=1000,
            height=400,
        )
    )

    chart = bars_chart + player_percent_chart


    chart.properties(
        width=1000,
        height=400,
        title="Player Count by Rank"
    )

    return chart


if __name__ == "__main__":
    # Example usage
    db_path = "../data/ranked_players.db"  # Replace with your database path
    db_path = os.path.abspath(db_path)
    try:
        chart = graph_player_count_by_rank(db_path, min_no_of_games=0)
        chart.display()
        chart.save("../graphs/player_count_by_rank.html")  # Save the chart as an HTML file
        print("Chart saved as player_count_by_rank.html")

        
    except Exception as e:
        print(f"An error occurred: {e}")


Data Description:
               count
count      24.000000
mean    50501.166667
std     31242.679362
min      5466.000000
25%     32195.750000
50%     42304.000000
75%     68101.500000
max    122705.000000
Total Players: 1212028


alt.LayerChart(...)

Chart saved as player_count_by_rank.html
